## Financial Fraud Classifier
#### Sam Berkson and Ben Puryear
#### CPSC 322 Final Project

In [ ]:
from mysklearn import myclassifiers, myevaluation, mypytable, myutils
import importlib

### Introduction)  
    * Why are we doing this in the first place?
        * Lots of real life applications:
            * Banking
            * Cyber Security
            * Data Science


![image](Unknown.jpeg)

    * Dataset
        * Our dataset originates from Kaggle, coming packaged as a csv file.  This csv file contains just over a million instances of different types of financial transactions from different accounts, and marks whether or not the transaction was flagged as fradulent or not, and whether or not the transaction was actually fradulent.  Our dataset contains 11 attributes:

        * step
            * Step maps to a number of hours, where 1 step is 1 hour.
        * type
            * This identifies the type of transaction.  It can be: CASH-IN; CASH-OUT; DEBIT; PAYMENT; and TRANSFER.
            * This could be a pretty useful attribute to use in classification.
        * amount
            * This is the amount of money transferred (in the local currency).
            * This can also be a pretty useful attribute to use in our classification.
        * nameOrig
            * This identifies the customer who initiated the transaction.
        * oldBalanceOrg
            * This is the initial balance before the transaction.
        * newBalanceOrig
            * This is the new balance after the transaction.
        * nameDest
            * This identifies the recipient of the transfer.
        * oldBalanceDest
            * This is the initial balance recipient before the transaction.
        * newBalanceDest
            * This is the new balance recipient after the transaction.
            * This can be a useful tool for classification when used in comparison to oldBalanceDest for any given instance.
        * isFraud
            * This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.
        * isFlaggedFraud
            * The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

Our classification goal is to correctly predict whether any given transaction is fradulent or not.

### EDA)

### Classification)

### Conclusion)